In [1]:
# require(data.table)
# require(bit64)
# require(dbscan)
# require(doParallel)
# require(rBayesianOptimization)
# path='../input/train_1/'
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event
from trackml.randomize import shuffle_hits

from sklearn.preprocessing import StandardScaler
import hdbscan as _hdbscan
from scipy import stats
from tqdm import tqdm

import time

from sklearn.cluster.dbscan_ import dbscan
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KDTree
import hdbscan
from bayes_opt import BayesianOptimization

# https://www.ellicium.com/python-multiprocessing-pool-process/
# http://sebastianraschka.com/Articles/2014_multiprocessing.html
from multiprocessing import Pool

In [2]:
import os
import time
import hdbscan as _hdbscan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [4]:
def preprocess(hits):        
    x = hits.x.values
    y = hits.y.values
    z = hits.z.values

    r = np.sqrt(x**2 + y**2 + z**2)
    hits['x2'] = x/r
    hits['y2'] = y/r

    r = np.sqrt(x**2 + y**2)
    hits['z2'] = z/r

    ss = StandardScaler()
    X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
#     for i, rz_scale in enumerate(self.rz_scales):
#         X[:,i] = X[:,i] * rz_scale

    return X

In [5]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#------------------------------------------------------

def make_counts(labels):
    
    
    _,reverse,count = np.unique(labels,return_counts=True,return_inverse=True)
    counts = count[reverse]
    counts[labels==0]=0
    
    return counts

def one_loop(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r, a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da 
    zr = z/r
    
    X = StandardScaler().fit_transform(np.column_stack([aa, aa/zr, zr, 1/zr, aa/zr + 1/zr]))
    _,l = dbscan(X, eps=0.0035, min_samples=1,)

    return l

def one_loop1(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def one_loop2(param):
    
    # <todo> tune your parameters or design your own features here! 
    
    i,m, x,y,z, d,r,r2,z2,a, a_start,a_step = param
    #print('\r %3d  %+0.8f '%(i,da), end='', flush=True)
    
    da = m*(a_start - (i*a_step))
    aa = a + np.sign(z)*z*da
#     if m == 1:
#         print(da)
    zr = z/r # this is cot(theta), 1/zr is tan(theta)
    theta = np.arctan2(r, z)
    ct = np.cos(theta)
    st = np.sin(theta)
    tt = np.tan(theta)
#     ctt = np.cot(theta)
    z2r = z2/r
    z2r2 = z2/r2
#     X = StandardScaler().fit_transform(df[['r2', 'theta_1', 'dip_angle', 'z2', 'z2_1', 'z2_2']].values)

    caa = np.cos(aa)
    saa = np.sin(aa)
    taa = np.tan(aa)
    ctaa = 1/taa
    
#     0.000005
    deps = 0.0000025
    X = StandardScaler().fit_transform(np.column_stack([caa, saa, tt, 1/tt]))
    l= DBSCAN(eps=0.0035+i*deps,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#     _,l = dbscan(X, eps=0.0035, min_samples=1,algorithm='auto')
    
    return l

def do_dbscan_predict(df):
    
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    r  = np.sqrt(x**2+y**2)
    d  = np.sqrt(x**2+y**2+z**2)
    a  = np.arctan2(y,x)

    x2 = df['x']/d
    y2 = df['y']/d
    z2 = df['z']/r

    r2 = np.sqrt(x2**2 + y2**2)
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    phi2 = np.arctan2(y2, x2)
    phi2_deg = np.degrees(np.arctan2(y2, x2))
    
    for angle in range(-180,180,1):
    
        df1 = df.loc[(df.phi_deg>(angle-1.0)) & (df.phi_deg<(angle+1.0))]
    
        x  = df1.x.values
        y  = df1.y.values
        z  = df1.z.values
        r  = np.sqrt(x**2+y**2)
        d  = np.sqrt(x**2+y**2+z**2)
        a  = np.arctan2(y,x)

        x2 = df1['x']/d
        y2 = df1['y']/d
        z2 = df1['z']/r
    
        r2 = np.sqrt(x2**2 + y2**2)
        theta= np.arctan2(r, z)
        theta1 = np.arctan2(r2, z2)

    
        tan_dip = phi/theta
        tan_dip1 = phi/z2
        z2_1 = 1/z2
        z2_2 = phi/z2 + 1/z2

        dip_angle = np.arctan2(z2, (np.sqrt(x2**2 +y2**2)) * np.arccos(x2/np.sqrt(x2**2 + y2**2)))
        dip_angle1 = np.arctan2(z, (np.sqrt(x**2 +y**2)) * np.arccos(x2/np.sqrt(x**2 + y**2)))
        scores = []

        a_start,a_step,a_num = 0.00100,0.0000095,150
    
        params  = [(i,m, x,y,z,d,r,r2,z2, a, a_start,a_step) for i in range(a_num) for m in [-1,1]]

        if 1: 
            pool = Pool(processes=1)
            ls   = pool.map( one_loop1, params )


        if 0:
            ls = [ one_loop(param) for param in  params ]


        ##------------------------------------------------

        num_hits=len(df)
        labels = np.zeros(num_hits,np.int32)
        counts = np.zeros(num_hits,np.int32)
        for l in ls:
            c = make_counts(l)
            idx = np.where((c-counts>0) & (c<20))[0]
            labels[idx] = l[idx] + labels.max()
            counts = make_counts(labels)
       

#     cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                          metric='braycurtis',cluster_selection_method='leaf',algorithm='best', 
#                          leaf_size=50)
    
#     X = preprocess(df)
#     l1 = pd.Series(labels)
#     labels = np.unique(l1)
   
# #   print(X.shape)
# #   print(len(labels_org))
# #   print(len(labels_org[labels_org ==0]))
# #   print(len(labels_org[labels_org ==-1]))
    
#     n_labels = 0
#     while n_labels < len(labels):
#         n_labels = len(labels)
#         max_len = np.max(l1)
#         s = list(l1[l1 == 0].keys())
#         X = X[s]
#         print(X.shape)
#         if X.shape[0] <= 1:
#             break
#         l = cl.fit_predict(X)+max_len
# #         print(len(l))
#         l1[l1 == 0] = l
#         labels = np.unique(l1)
    
    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

## reference----------------------------------------------
def do_dbscan0_predict(df):
    x = df.x.values
    y = df.y.values
    z = df.z.values
    r = np.sqrt(x**2+y**2)
    d = np.sqrt(x**2+y**2+z**2)

    X = StandardScaler().fit_transform(np.column_stack([
        x/d, y/d, z/r]))
    _,labels = dbscan(X,
                eps=0.0075,
                min_samples=1,
                algorithm='auto',
                n_jobs=-1)

    #labels = hdbscan(X, min_samples=1, min_cluster_size=5, cluster_selection_method='eom')

    return labels

def extend(submission,hits):
    df = submission.merge(hits,  on=['hit_id'], how='left')
#     df = submission.append(hits)
#     print(df.head())
#     dz = 0
#     df.z = df.z + dz
    df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
    df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
    df = df.assign(arctan2 = np.arctan2(df.z, df.r))

    for angle in range(-180,180,1):

        print ('\r %f'%angle, end='',flush=True)
        #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
        df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

        min_num_neighbours = len(df1)
        if min_num_neighbours<4: continue

        hit_ids = df1.hit_id.values
        x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
        r  = (x**2 + y**2)**0.5
        r  = r/1000
        a  = np.arctan2(y,x)
        tree = KDTree(np.column_stack([a,r]), metric='euclidean')

        track_ids = list(df1.track_id.unique())
        num_track_ids = len(track_ids)
        min_length=3

        for i in range(num_track_ids):
            p = track_ids[i]
            if p==0: continue

            idx = np.where(df1.track_id==p)[0]
            if len(idx)<min_length: continue

            if angle>0:
                idx = idx[np.argsort( z[idx])]
            else:
                idx = idx[np.argsort(-z[idx])]


            ## start and end points  ##
            idx0,idx1 = idx[0],idx[-1]
            a0 = a[idx0]
            a1 = a[idx1]
            r0 = r[idx0]
            r1 = r[idx1]

            da0 = a[idx[1]] - a[idx[0]]  #direction
            dr0 = r[idx[1]] - r[idx[0]]
            direction0 = np.arctan2(dr0,da0) 

            da1 = a[idx[-1]] - a[idx[-2]]
            dr1 = r[idx[-1]] - r[idx[-2]]
            direction1 = np.arctan2(dr1,da1) 


            ## extend start point
            ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)
            direction = np.arctan2(r0-r[ns],a0-a[ns])
            ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

            ## extend end point
            ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
            ns = np.concatenate(ns)

            direction = np.arctan2(r[ns]-r1,a[ns]-a1)
            ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 

            for n in ns:
                df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
    #print ('\r')
#     df = df[['particle_id', 'weight', 'event_id', 'hit_id', 'track_id']]
    df = df[['event_id', 'hit_id', 'track_id']]
    return df

In [6]:
def trackML(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    print(Niter)
    Niter = Niter.astype(np.int32)
    print(Niter)
    print(w1, w2, w3, Niter)
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(1,X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [7]:
def trackML2(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.000005*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(1,X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [8]:
candidates = []
def trackML3(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r]))
#         print(X.shape)
        w4 = 0.1470
        cx = [w1,w1,w2,w3,w4]
        for jj in range(1,X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        
        
        track_ids = np.unique(l)
        track_ids = track_ids[track_ids!=0]
        neigbour = [ np.where(l==t)[0] for t in track_ids]

        unique,inverse,c = np.unique(l,return_counts=True,return_inverse=True)
        unique = unique[unique!=0]
        c = c[inverse]
        c[l==0] = 0

        for u in unique:
            candidate = np.where(l==u)[0]
            candidates.append(candidate)
                
#         ls.append(l)
        
#     num_hits=len(df)
#     labels = np.zeros(num_hits,np.int32)
#     counts = np.zeros(num_hits,np.int32)
#     for l in ls:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)
        
    return candidates

In [9]:
def trackML4(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    mm = 1
    ls = []
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z3]))
#         print(X.shape)
        
        cx = [w1,w1,w2,w3]
        for jj in range(1,X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        
        
        track_ids = np.unique(l)
        track_ids = track_ids[track_ids!=0]
        neigbour = [ np.where(l==t)[0] for t in track_ids]

        unique,inverse,c = np.unique(l,return_counts=True,return_inverse=True)
        unique = unique[unique!=0]
        c = c[inverse]
        c[l==0] = 0

        for u in unique:
            candidate = np.where(l==u)[0]
            candidates.append(candidate)
                
#         ls.append(l)
        
#     num_hits=len(df)
#     labels = np.zeros(num_hits,np.int32)
#     counts = np.zeros(num_hits,np.int32)
#     for l in ls:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)
        
    return candidates

In [10]:
def trackML5(df, w1, w2, w3, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    Niter=179
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r]))
#         print(X.shape)
        w4 = 0.1470
        cx = [w1,w1,w2,w3,w4]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_

        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [11]:
def trackML6(df, w1, w2, w3, w5, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    theta = np.arctan2(rt, z)
    tt = np.tan(theta)
    
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = 0.1470
    Niter=179
    
    mm = 1
    ls = []
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
        a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        deps = 0.0000025
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, rt/r, z3]))
#         print(X.shape)
        w4 = 0.1470
        cx = [w1,w1,w2,w4, w5]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        l= DBSCAN(eps=0.0035,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_

        ls.append(l)
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [12]:
import hdbscan
def trackML31(df, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12, w13, Niter):
    x  = df.x.values
    y  = df.y.values
    z  = df.z.values
    
#     dz = -5.5
#     z = z - 5.5
    
    rt  = np.sqrt(x**2+y**2)
    r  = np.sqrt(x**2+y**2+z**2)
    a0  = np.arctan2(y,x)

    x2 = x/r
    y2 = y/r
    
    
    phi = np.arctan2(y, x)
    phi_deg= np.degrees(np.arctan2(y, x))
    
    
    z1 = z/rt
    z2 = z/r
    
    z3 = np.log1p(abs(z/r))*np.sign(z)
    
    x1 = x/rt
    y1 = y/rt
    
    y3 = np.log1p(abs(y/r))*np.sign(y)
    
    theta = np.arctan2(rt, z)
    theta_deg = np.degrees(np.arctan2(rt, z))
    tt = np.tan(theta_deg)
    
    z4 = np.sqrt(abs(z/rt))
    
    x4 = np.sqrt(abs(x/r))
    y4 = np.sqrt(abs(y/r))
    
    mm = 1
    ls = []
    
#     print(Niter)
#     Niter = Niter.astype(np.int32)
#     print(Niter)
#     print(w1, w2, w3, Niter)
    
    for ii in range(Niter):
        mm = mm * (-1)
#         print(np.pi)
#         a1 = a0+mm*(rt+0.000045*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+0.0000075*rt**2)/1000*(ii/2)/180*np.pi
        a1 = a0+mm*(rt+ 0.0000145*rt**2)/1000*(ii/2)/180*np.pi
#         a1 = a0+mm*(rt+ 0.0000005*rt**2)/1000*(ii/2)/180*np.pi
#         print(a0, a1)
        saa = np.sin(a1)
        caa = np.cos(a1)
        
        t1 = theta+mm*(rt+ 0.8435*rt**2)/1000*(ii/2)/180*np.pi
        ctt = np.cos(t1)
        stt = np.sin(t1)
        ttt = np.tan(t1)
        deps = 0.0000025
        
        
        
        r0Inv = 2. * np.cos(a1 - t1) / r
        a2 = np.nan_to_num(np.arccos(r * r0Inv * 0.5))
        
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, z2, rt/r, x/r, z3, ctt, stt]))
#         X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1, y/r, z3, y1, y3, 
#                                                             ctt, stt, z4, x4, y4]))
    
        X = StandardScaler().fit_transform(np.column_stack([caa, saa, z1]))
#         print(X.shape)
        
#         cx = [w1,w1,w2,w3, w4, w5, w6, w7, w7]
#         cx = [w1,w1,w2, w6, w7, w8, w9, w10, w10, w11, w12, w13]
        cx = [w1,w1,w2, w3]
        for jj in range(X.shape[1]): 
             X[:,jj] = X[:,jj]*cx[jj]
        deps = 0.0000025
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,cluster_selection_method='leaf',metric='euclidean', leaf_size=50)
#         l = cl.fit_predict(X)+1
        
        l= DBSCAN(eps=0.004,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
#         l= DBSCAN(eps=w7,min_samples=1,metric='euclidean',n_jobs=8).fit(X).labels_
        ls.append(l)
    
        
    num_hits=len(df)
    labels = np.zeros(num_hits,np.int32)
    counts = np.zeros(num_hits,np.int32)
    for l in ls:
        c = make_counts(l)
        idx = np.where((c-counts>0) & (c<20))[0]
        labels[idx] = l[idx] + labels.max()
        counts = make_counts(labels)
        
    return labels

In [13]:
######################## 
### working function ###
# ########################
# trackML <- function(dfh,w1,w2,w3,Niter){
#   dfh[,s1:=hit_id]
#   dfh[,N1:=1L] 
#   dfh[,r:=sqrt(x*x+y*y+z*z)]
#   dfh[,rt:=sqrt(x*x+y*y)]
#   dfh[,a0:=atan2(y,x)]
#   dfh[,z1:=z/rt]
#   dfh[,z2:=z/r]
#   mm     <-  1
#   for (ii in 0:Niter) {
#     mm <- mm*(-1)
#     dfh[,a1:=a0+mm*(rt+0.000005*rt^2)/1000*(ii/2)/180*pi]
#     dfh[,sina1:=sin(a1)]
#     dfh[,cosa1:=cos(a1)]
#     dfs=scale(dfh[,.(sina1,cosa1,z1,z2)])
# 	cx <- c(w1,w1,w2,w3)
#     for (jj in 1:ncol(dfs)) dfs[,jj] <- dfs[,jj]*cx[jj]
#     res=dbscan(dfs,eps=0.0035,minPts = 1)
#     dfh[,s2:=res$cluster]
#     dfh[,N2:=.N, by=s2]
#     maxs1 <- max(dfh$s1)
#     dfh[,s1:=ifelse(N2>N1 & N2<20,s2+maxs1,s1)]
#     dfh[,s1:=as.integer(as.factor(s1))]
#     dfh[,N1:=.N, by=s1]    
#   }
#   return(dfh$s1)
# }

In [14]:
def Fun4BO(w1,w2,w3,Niter):
    track_id = trackML(hits, w1,w2,w3,Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [15]:
def Fun4BO1(w1,w2,w3,Niter):
    candidates = trackML3(hits, w1,w2,w3,Niter)
#     candidates = trackML4(hits, w1,w2,w3,Niter)
    
    #sort
    count = np.array([len(candidate) for candidate in candidates])
    sort = np.argsort(-count)
    candidates = [candidates[s] for s in sort]
    
    #show
    max_label=1
    N =len(hits)
    label = np.zeros(N,np.int32)
    count = np.zeros(N,np.int32)

         
#     num_hits=len(hits)
#     labels = np.zeros(num_hits,np.int32)
#     counts = np.zeros(num_hits,np.int32)
#     for l in ls:
#         c = make_counts(l)
#         idx = np.where((c-counts>0) & (c<20))[0]
#         labels[idx] = l[idx] + labels.max()
#         counts = make_counts(labels)
        
    layer_id = hits['layer_id'].values.astype(np.float32)
    
    z = hits.z.values
    for candidate in candidates:
        n = candidate
        
        L = len(n)
        if L > 20: continue
#         print(L)
#         print(n)

        #---- filtering (secret sauce) ----------
        #if L<3: continue
        n = n[np.argsort(np.fabs(z[n]))]
#         print(n)
        
        layer_id0 = layer_id[n[:-1]]
        layer_id1 = layer_id[n[1: ]]
        ld = layer_id1-layer_id0
#         if np.any(ld>2): continue

        m = count[n].max()
        if L<m: continue

        #---- filtering ----------------------

        count[n]=L
        label[n]=max_label
        max_label += 1
        
    track_id = label
#     track_id = labels
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [16]:
def Fun4BO2(w1, w2, w3):
    
#     w1 = 1.1932215111905984
#     w2 = 0.39740553885387364
#     w1 = 0.3
#     w2 = 0.3
#     w3 = 0.3512647720585538
#     w4 = 0.1470
#     w5 = 0.01201
#     w2 = 0.4893
#     w3 = 0.2697
#     w5 = 1
    w6 = 0.0003864
    w7 = 0.0205
    w8 = 0.0049
    w9 = 0.00121
    w10 = 1.4930496676654575e-05
    w11 = 0.0318
    w12 = 0.000435
    w13 = 0.00038
    Niter=179
    Niter=247
    w4 = 0
    w5 = 0
    print(w1, w2, w3)
#     Niter = Niter.astype(np.int32)
    track_id = trackML31(hits, w1,w2,w3,w4,w5,w6,w7,w8,w9,w10, w11, w12, w13, Niter)
    
    sum_score=0
    sum = 0
    submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
        data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
    ).astype(int)
    for i in range(8):
        submission = extend(submission,hits)
        score = score_event(truth, submission)
        print('[%2d] score : %0.8f'%(i, score))
        sum_score += score
        sum += 1

    print('--------------------------------------')
    sc = sum_score/sum
    print(sc)
    
    return sc

In [17]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    hits.z = hits.z - 5.5
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    w1 = 1.1932215111905984
    w2 = 0.39740553885387364
    w3 = 0.3512647720585538
    w4 = [0.1, 0.2] # 0.1470 -> 0.55690
    w4 = 0.1470
    w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    
    Niter = 179
    Niter = 247
    w5 = 0.01
#     for w6 in [0.012, 0.01201, 0.01202, 0.01203, 0.01204, 0.01205, 0.01206, 0.01207, 0.01208, 0.01209, 0.0121]:
    EPS = 1e-12
    w6 = [0.001, 1.2]
    w6 = 0.0205
    w8 = [0.00008, 0.0005]
#     for w8 in np.arange(0.00008, 0.00015, 0.000005):
#         print(w8)
#         Fun4BO2(w8)
#     Fun4BO2(0)
#     for w1 in np.arange(0.5, 3., 0.05):
#         print(w1)
#         Fun4BO2(w1)
#     Niter = [240, 480]
#     w12 = [0.00001, 1.2]
    w1 = [0.00001, 1.2]
    w2 = [0.00001, 1.2]
    w3 = [0.00001, 1.2]
    w4 = [0.01, 1.2] # 0.1470 -> 0.55690
#     w4 = 0.1470
    w5 = [0.001, 1.2] # 0.7781 -> 0.55646, 0.7235 + N = 247 => 0.56025
    
    bo = BayesianOptimization(Fun4BO2,pbounds = {'w1':w1, 'w2':w2, 'w3':w3})
    bo.maximize(init_points = 40, n_iter = 5, acq = "ucb", kappa = 2.576)

# x/y: 7 | 06m30s |    0.55302 |    0.0100 | 
# x/y: 0.001: 0.55949
# x/y: 0.0001: 0.55949
# x/y: 0.002: 0.55959
# x/y: 0.003: 0.55915
# x/y: 0.0025: 0.55925
# x/y: 0.0015: 0.55953
# x/r: 0.0015: 0.56186
# x/r: 0.002: 0.56334
# x/r: 0.0025: 0.563989
# x/r: 0.003: 0.56447
# x/r: 0.01: 0.569822
# x/r: 0.015: 0.56940
# x/r: 0.012: 0.5719
# x/r: 0.01201: 0.57192
# 1.4499999999999993e-05 * rt**2: 0.5720702851970194
# 0.0000145
# z3: 10 | 07m12s |    0.57208 |    0.0205 | 
# count: 19: 0.572567, 17: 0.57263
# ctt, stt after change:  2 | 07m56s |    0.57345 |    0.0001 | (0.00010567777727496665)
# x4: 25 | 09m42s |    0.57359 |    0.0002 | (0.000206214286412982)
# x4: 0.000435 (0.5737387485278771) (x4 = np.sqrt(abs(x/r)))
# w13: 00038 (ctt,stt): 0.5737528800479372
# dz = dz - 5.5 (0.5179179093800803)
# w1 = 0.3 (0.557986859422024)
# test dz = -5.5 reweight ( 25 | 07m32s |    0.49190 |    1.2000 |    0.4893 |    0.2697 | )

Initialization
-----------------------------------------------------------------
 Step |   Time |      Value |        w1 |        w2 |        w3 | 
0.7605531331537437 0.677142033373525 0.7455542471105937
 179.0000000[ 0] score : 0.37429625
 179.0000000[ 1] score : 0.37868123
 179.0000000[ 2] score : 0.37999092
 179.0000000[ 3] score : 0.38039901
 179.0000000[ 4] score : 0.38043991
 179.0000000[ 5] score : 0.38067735
 179.0000000[ 6] score : 0.38056997
 179.0000000[ 7] score : 0.38051948
--------------------------------------
0.37944676460519317
    1 | 06m05s |    0.37945 |    0.7606 |    0.6771 |    0.7456 | 
0.18606420134268056 0.8504098099487012 0.9498626223176079
 179.0000000[ 0] score : 0.24411737
 179.0000000[ 1] score : 0.25092582
 179.0000000[ 2] score : 0.25391608
 179.0000000[ 3] score : 0.25537107
 179.0000000[ 4] score : 0.25563646
 179.0000000[ 5] score : 0.25554025
 179.0000000[ 6] score : 0.25577219
 179.0000000[ 7] score : 0.25551923
------------------------------------

 179.0000000[ 0] score : 0.41171750
 179.0000000[ 1] score : 0.41636355
 179.0000000[ 2] score : 0.41773523
 179.0000000[ 3] score : 0.41871700
 179.0000000[ 4] score : 0.41815807
 179.0000000[ 5] score : 0.41860207
 179.0000000[ 6] score : 0.41832808
 179.0000000[ 7] score : 0.41865707
--------------------------------------
0.4172848228226333
   18 | 06m56s |    0.41728 |    0.9658 |    0.8306 |    0.1153 | 
0.3422280434514052 1.0376386815511913 1.0469873351857784
 179.0000000[ 0] score : 0.29553486
 179.0000000[ 1] score : 0.29953419
 179.0000000[ 2] score : 0.30214210
 179.0000000[ 3] score : 0.30287764
 179.0000000[ 4] score : 0.30339972
 179.0000000[ 5] score : 0.30354649
 179.0000000[ 6] score : 0.30369130
 179.0000000[ 7] score : 0.30347961
--------------------------------------
0.30177573992681467
   19 | 07m04s |    0.30178 |    0.3422 |    1.0376 |    1.0470 | 
0.2743892106027268 0.6069122888860817 1.1567203005327602
 179.0000000[ 0] score : 0.26366095
 179.0000000[ 1] score 

 179.0000000[ 3] score : 0.23805318
 179.0000000[ 4] score : 0.23850926
 179.0000000[ 5] score : 0.23855978
 179.0000000[ 6] score : 0.23848086
 179.0000000[ 7] score : 0.23852388
--------------------------------------
0.23718371062415258
   35 | 06m39s |    0.23718 |    0.8937 |    0.1433 |    0.1371 | 
0.8227094759663155 0.7836621638428164 0.06621004970297319
 179.0000000[ 0] score : 0.38874145
 179.0000000[ 1] score : 0.39432739
 179.0000000[ 2] score : 0.39505946
 179.0000000[ 3] score : 0.39622192
 179.0000000[ 4] score : 0.39514218
 179.0000000[ 5] score : 0.39585982
 179.0000000[ 6] score : 0.39557961
 179.0000000[ 7] score : 0.39618045
--------------------------------------
0.39463903628116226
   36 | 06m29s |    0.39464 |    0.8227 |    0.7837 |    0.0662 | 
1.187885138634084 0.3958004562484892 0.5493224633223163
 179.0000000[ 0] score : 0.36569054
 179.0000000[ 1] score : 0.36838445
 179.0000000[ 2] score : 0.37046845
 179.0000000[ 3] score : 0.37021887
 179.0000000[ 4] score

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


1.2 1.1999999977582505 0.6818430459024705
 179.0000000[ 0] score : 0.42493167
 179.0000000[ 1] score : 0.43403161
 179.0000000[ 2] score : 0.43587176
 179.0000000[ 3] score : 0.43741161
 179.0000000[ 4] score : 0.43679732
 179.0000000[ 5] score : 0.43724076
 179.0000000[ 6] score : 0.43673379
 179.0000000[ 7] score : 0.43717047
--------------------------------------
0.4350236239094506
   43 | 06m52s |    0.43502 |    1.2000 |    1.2000 |    0.6818 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00025726]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


1.1999999995727924 0.8683360824537414 1.199999999946872
 179.0000000[ 0] score : 0.44174364
 179.0000000[ 1] score : 0.44788029
 179.0000000[ 2] score : 0.44875044
 179.0000000[ 3] score : 0.44942367
 179.0000000[ 4] score : 0.44887624
 179.0000000[ 5] score : 0.44911275
 179.0000000[ 6] score : 0.44915037
 179.0000000[ 7] score : 0.44917700
--------------------------------------
0.44801429918354085
   44 | 06m52s |    0.44801 |    1.2000 |    0.8683 |    1.2000 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


1.1999999870502647 0.8881837616074401 0.6941750067498976
 179.0000000[ 0] score : 0.44185409
 179.0000000[ 1] score : 0.44777363
 179.0000000[ 2] score : 0.44882237
 179.0000000[ 3] score : 0.44926021
 179.0000000[ 4] score : 0.44909567
 179.0000000[ 5] score : 0.44879711
 179.0000000[ 6] score : 0.44902045
 179.0000000[ 7] score : 0.44906304
--------------------------------------
0.44796082004081306
   45 | 06m44s |    0.44796 |    1.2000 |    0.8882 |    0.6942 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [ ]:
len(t)

In [54]:
len(t[t==0])

2

In [52]:
len(t[t == 10904474])

13

In [20]:
import Cluster_Ensembles as CE
# def Fun4BO2(w5):

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.1439
Niter=179

track_id1 = trackML5(hits, w1,w2,w3,Niter) # 0.55613 indiv
track_id2 = trackML6(hits, w1,w2,w3,w5, Niter) # 0.55664 indiv

num_hits=len(hits)
labels = np.zeros((2,num_hits),np.int32)

labels[0] = track_id1
labels[1] = track_id2
track_id = CE.cluster_ensembles(labels, verbose = True, N_clusters_max = 9800)

sum_score=0
sum = 0
submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
    data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
).astype(int)
for i in range(8):
    submission = extend(submission,hits)
    score = score_event(truth, submission)
    print('[%2d] score : %0.8f'%(i, score))
    sum_score += score
    sum += 1

print('--------------------------------------')
sc = sum_score/sum
print(sc)
# 9000 ==> 0.49
# 9100 ==> 0.4987
# 10000 ==> 0.52
# 17000 ==> 0.48


INFO: Cluster_Ensembles: cluster_ensembles: due to a rather large number of cells in your data-set, using only 'HyperGraph Partitioning Algorithm' (HGPA) and 'Meta-CLustering Algorithm' (MCLA) as ensemble consensus functions.


*****
INFO: Cluster_Ensembles: HGPA: consensus clustering using HGPA.

#
INFO: Cluster_Ensembles: wgraph: writing wgraph_HGPA.
INFO: Cluster_Ensembles: wgraph: 113998 vertices and 78748 non-zero hyper-edges.
#

#
INFO: Cluster_Ensembles: sgraph: calling shmetis for hypergraph partitioning.
INFO: Cluster_Ensembles: sgraph: (hyper)-graph partitioning completed; loading wgraph_HGPA.part.9100
#

INFO: Cluster_Ensembles: checkcl: the vector cluster labellings provided is not a dense integer mapping.
INFO: Cluster_Ensembles: checkcl: brought modification to this vector so that its labels range from 0 to 39811, included.


INFO: Cluster_Ensembles: checkcl: the vector cluster labellings provided is not a dense integer mapping.
INFO: Cluster_Ensembles: checkcl: brought 

In [155]:
hits.particle_id.nunique()

9728

In [17]:
print(len(set(track_id)))
print(len(set(track_id1)))
print(len(set(track_id2)))

10000
39812
38936


In [149]:
import Cluster_Ensembles as CE
# def Fun4BO2(w5):


w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.1439
Niter=179

track_id1 = trackML5(hits, w1,w2,w3,Niter)
track_id2 = trackML6(hits, w1,w2,w3,w5, Niter)

num_hits=len(hits)
labels = np.zeros((2,num_hits),np.int32)

labels[0] = track_id1
labels[1] = track_id2
track_id = CE.cluster_ensembles(labels, verbose = True, N_clusters_max = 20000)

sum_score=0
sum = 0
submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
    data=np.column_stack(([int(event_id),]*len(hits), hits.hit_id.values, track_id))
).astype(int)
for i in range(8):
    submission = extend(submission,hits)
    score = score_event(truth, submission)
    print('[%2d] score : %0.8f'%(i, score))
    sum_score += score
    sum += 1

print('--------------------------------------')
sc = sum_score/sum
print(sc)
    
#     return sc


INFO: Cluster_Ensembles: cluster_ensembles: due to a rather large number of cells in your data-set, using only 'HyperGraph Partitioning Algorithm' (HGPA) and 'Meta-CLustering Algorithm' (MCLA) as ensemble consensus functions.


*****
INFO: Cluster_Ensembles: HGPA: consensus clustering using HGPA.

#
INFO: Cluster_Ensembles: wgraph: writing wgraph_HGPA.
INFO: Cluster_Ensembles: wgraph: 113998 vertices and 78748 non-zero hyper-edges.
#

#
INFO: Cluster_Ensembles: sgraph: calling shmetis for hypergraph partitioning.
INFO: Cluster_Ensembles: sgraph: (hyper)-graph partitioning completed; loading wgraph_HGPA.part.20000
#

INFO: Cluster_Ensembles: checkcl: the vector cluster labellings provided is not a dense integer mapping.
INFO: Cluster_Ensembles: checkcl: brought modification to this vector so that its labels range from 0 to 39811, included.


INFO: Cluster_Ensembles: checkcl: the vector cluster labellings provided is not a dense integer mapping.
INFO: Cluster_Ensembles: checkcl: brought

In [148]:
labels[0] = track_id1
labels[1] = track_id2
track_id = CE.cluster_ensembles(labels, verbose = True, N_clusters_max = 20000)

NameError: name 'track_id1' is not defined

In [146]:
#######################################
# function for Bayessian Optimization #
#   (needs lists: Score and Pred)     #
#######################################
# Fun4BO <- function(w1,w2,w3,Niter) { 
#    dfh$s1 <- trackML(dfh,w1,w2,w3,Niter)
#    sub=data.table(event_id=nev,hit_id=dfh$hit_id,track_id=dfh$s1)
#    sc <- score(sub,dft)
#    list(Score=sc,Pred=0)
# }

In [147]:
w1 = 1.0561046005761139
w2 = 0.5351622558071004
w3 = 0.3762123539827661
Niter = 184

# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
#     w1 = [0.9, 1.2]
#     w2 = [0.3, 0.7]
#     w3 = [0.1, 0.4]
#     Niter = [140, 190]
    w5 = [0.1, 1.3]
    w5 = 0.1439
    
#     bo = BayesianOptimization(Fun4BO2,pbounds = {'w5':w5})
#     bo.maximize(init_points = 5, n_iter = 10, acq = "ucb", kappa = 2.576)
    Fun4BO2(w5)
    # 0.55613
    # z3: 3 | 06m28s |    0.55615 |    0.2167 | 
    # z3: 3 | 06m29s |    0.55665 |    0.1439 | 


INFO: Cluster_Ensembles: cluster_ensembles: due to a rather large number of cells in your data-set, using only 'HyperGraph Partitioning Algorithm' (HGPA) and 'Meta-CLustering Algorithm' (MCLA) as ensemble consensus functions.


*****
INFO: Cluster_Ensembles: HGPA: consensus clustering using HGPA.

#
INFO: Cluster_Ensembles: wgraph: writing wgraph_HGPA.
INFO: Cluster_Ensembles: wgraph: 113998 vertices and 78748 non-zero hyper-edges.
#

#
INFO: Cluster_Ensembles: sgraph: calling shmetis for hypergraph partitioning.


FileNotFoundError: [Errno 2] No such file or directory: 'wgraph_HGPA.part.113998'

In [130]:
w1 = 1.0561046005761139
w2 = 0.5351622558071004
w3 = 0.3762123539827661
Niter = 184

# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
#     w1 = [0.9, 1.2]
#     w2 = [0.3, 0.7]
#     w3 = [0.1, 0.4]
#     Niter = [140, 190]
    
    
#     bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
#     bo.maximize(init_points = 5, n_iter = 10, acq = "ucb", kappa = 2.576)
    Fun4BO2(w1,w2,w3,Niter)
    # 0.55613

 179.0000000[ 0] score : 0.54945841
 179.0000000[ 1] score : 0.55564882
 179.0000000[ 2] score : 0.55701564
 179.0000000[ 3] score : 0.55734112
 179.0000000[ 4] score : 0.55756978
 179.0000000[ 5] score : 0.55762537
 179.0000000[ 6] score : 0.55712773
 179.0000000[ 7] score : 0.55722315
--------------------------------------
0.5561262528478478


In [60]:
# with candidates code
Fun4BO1(w1,w2,w3,Niter)

 179.0000000[ 0] score : 0.47534472
 179.0000000[ 1] score : 0.49137631
 179.0000000[ 2] score : 0.49533204
 179.0000000[ 3] score : 0.49757213
 179.0000000[ 4] score : 0.49750988
 179.0000000[ 5] score : 0.49761983
 179.0000000[ 6] score : 0.49765447
 179.0000000[ 7] score : 0.49801465
--------------------------------------
0.4938030039598508


0.4938030039598508

In [72]:
Fun4BO1(w1,w2,w3,Niter)

 179.0000000[ 0] score : 0.53918686
 179.0000000[ 1] score : 0.54427397
 179.0000000[ 2] score : 0.54658418
 179.0000000[ 3] score : 0.54869202
 179.0000000[ 4] score : 0.54830880
 179.0000000[ 5] score : 0.54871004
 179.0000000[ 6] score : 0.54773057
 179.0000000[ 7] score : 0.54865996
--------------------------------------
0.5465183024481679


0.5465183024481679

In [78]:
Fun4BO1(w1,w2,w3,Niter)

 179.0000000[ 0] score : 0.53992070
 179.0000000[ 1] score : 0.54478272
 179.0000000[ 2] score : 0.54703642
 179.0000000[ 3] score : 0.54930453
 179.0000000[ 4] score : 0.54886812
 179.0000000[ 5] score : 0.54933398
 179.0000000[ 6] score : 0.54834621
 179.0000000[ 7] score : 0.54928390
--------------------------------------
0.5471095711394368


0.5471095711394368

In [19]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 5, n_iter = 10, acq = "ucb", kappa = 2.576)
    

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
146.70094267695322
146
1.0067149120750614 0.3292354279174612 0.30928510023285705 146
 179.0000000[ 0] score : 0.52168604
 179.0000000[ 1] score : 0.52668095
 179.0000000[ 2] score : 0.52842130
 179.0000000[ 3] score : 0.52965712
 179.0000000[ 4] score : 0.52915636
 179.0000000[ 5] score : 0.53008617
 179.0000000[ 6] score : 0.52904835
 179.0000000[ 7] score : 0.53013357
--------------------------------------
0.5281087329209931
    1 | 04m48s |    0.52811 |  146.7009 |    1.0067 |    0.3292 |    0.3093 | 
173.80001205446075
173
1.1676239153219412 0.42580395820724126 0.1491345652690207 173
 179.0000000[ 0] score : 0.51685554
 179.0000000[ 1] score : 0.52219516
 179.0000000[ 2] score : 0.52503955
 179.0000000[ 3] score : 0.52620415
 179.0000000[ 4] score : 0.52620623
 179.0000000[ 5] score : 0.52738113
 179.0000000[ 6] 

/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.28499297e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 57, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


Bayesian Optimization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
189.99999999875536
189
0.9 0.7 0.4 189
 179.0000000[ 0] score : 0.53311983
 179.0000000[ 1] score : 0.53869670
 179.0000000[ 2] score : 0.54013250
 179.0000000[ 3] score : 0.54135956
 179.0000000[ 4] score : 0.54060826
 179.0000000[ 5] score : 0.54118114
 179.0000000[ 6] score : 0.54092007
 179.0000000[ 7] score : 0.54098840
--------------------------------------
0.5396258082150559
    6 | 05m49s |    0.53963 |  190.0000 |    0.9000 |    0.7000 |    0.4000 | 


/home/watts/anaconda3/envs/tmlf/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([2.56714723e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 48, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


140.00000000001353
140
1.1999999999885052 0.7 0.1 140
 179.0000000[ 0] score : 0.51782841
 179.0000000[ 1] score : 0.52276632
 179.0000000[ 2] score : 0.52440509
 179.0000000[ 3] score : 0.52346236
 179.0000000[ 4] score : 0.52455302
 179.0000000[ 5] score : 0.52538912
 179.0000000[ 6] score : 0.52506728
 179.0000000[ 7] score : 0.52483711
--------------------------------------
0.5235385889152744
    7 | 05m08s |    0.52354 |  140.0000 |    1.2000 |    0.7000 |    0.1000 | 
161.86828236370445
161
0.9000000146281805 0.7 0.10000001462818049 161
 179.0000000[ 0] score : 0.51547569
 179.0000000[ 1] score : 0.52040849
 179.0000000[ 2] score : 0.52278077
 179.0000000[ 3] score : 0.52365546
 179.0000000[ 4] score : 0.52360893
 179.0000000[ 5] score : 0.52421784
 179.0000000[ 6] score : 0.52413636
 179.0000000[ 7] score : 0.52433024
--------------------------------------
0.5223267230888617
    8 | 05m21s |    0.52233 |  161.8683 |    0.9000 |    0.7000 |    0.1000 | 
179.64124366237593
179
0.9

In [15]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')
    
    w1 = [0.9, 1.2]
    w2 = [0.3, 0.7]
    w3 = [0.1, 0.4]
    Niter = [140, 190]
    
    
    bo = BayesianOptimization(Fun4BO,pbounds = {'w1':w1,'w2':w2,'w3':w3,'Niter':Niter})
    bo.maximize(init_points = 3, n_iter = 20, acq = "ucb", kappa = 2.576)
    

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |     Niter |        w1 |        w2 |        w3 | 
184.7827709130419
184
1.0561046005761139 0.5351622558071004 0.3762123539827661 184
 179.0000000[ 0] score : 0.53942962
 179.0000000[ 1] score : 0.54606243
 179.0000000[ 2] score : 0.54780086
 179.0000000[ 3] score : 0.54875295
 179.0000000[ 4] score : 0.54815666
 179.0000000[ 5] score : 0.54860441
 179.0000000[ 6] score : 0.54801785
 179.0000000[ 7] score : 0.54881484
--------------------------------------
0.5469549540012781
    1 | 05m08s |    0.54695 |  184.7828 |    1.0561 |    0.5352 |    0.3762 | 
145.0794227877262
145
1.1353520277193074 0.658195945930628 0.24032322705809864 145
 179.0000000[ 0] score : 0.52382250
 179.0000000[ 1] score : 0.52837456
 179.0000000[ 2] score : 0.53034994
 179.0000000[ 3] score : 0.53139514
 179.0000000[ 4] score : 0.53153080
 179.0000000[ 5] score : 0.53243534
 179.0000000[ 6] scor

 179.0000000[ 2] score : 0.45956269
 179.0000000[ 3] score : 0.46153693
 179.0000000[ 4] score : 0.46225305
 179.0000000[ 5] score : 0.46312552
 179.0000000[ 6] score : 0.46227716
 179.0000000[ 7] score : 0.46323152
--------------------------------------
0.46055974873573446
   17 | 05m21s |    0.46056 |  186.4388 |    1.2000 |    0.3000 |    0.1000 | 
175.93802446513746
175
1.2 0.7 0.4 175
 179.0000000[ 0] score : 0.52847339
 179.0000000[ 1] score : 0.53424707
 179.0000000[ 2] score : 0.53617756
 179.0000000[ 3] score : 0.53704935
 179.0000000[ 4] score : 0.53663841
 179.0000000[ 5] score : 0.53773147
 179.0000000[ 6] score : 0.53728202
 179.0000000[ 7] score : 0.53720027
--------------------------------------
0.5355999427729057
   18 | 05m21s |    0.53560 |  175.9380 |    1.2000 |    0.7000 |    0.4000 | 
164.75000708655898
164
0.9 0.7 0.4 164
 179.0000000[ 0] score : 0.52636347
 179.0000000[ 1] score : 0.53308124
 179.0000000[ 2] score : 0.53533614
 179.0000000[ 3] score : 0.53587774

In [13]:
# 0.54695
w1 = 1.0561046005761139
w2 = 0.5351622558071004
w3 = 0.3762123539827661
Niter = 184

# 172.3593 |    1.0579 |    0.5147 |    0.3853

In [17]:
w1 = 1.1932215111905984
w2 = 0.39740553885387364
w3 = 0.3512647720585538
w4 = 0.1470
w5 = 0.01201
w6 = 0.0003864
w7 = 0.0205
w8 = 0.0049
w9 = 0.00121
w10 = 1.4930496676654575e-05
Niter = 247

In [18]:
def run_make_submission():

    data_dir = '../data/test'


    tic = t = time.time()
    event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)

    if 1:
        submissions = []
        for i,event_id in enumerate(event_ids):
            hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
            cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)

            track_id = trackML31(hits, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, Niter)
            #track_id = back_fit(track_id,hits)

            toc =  time.time()
            print('\revent_id : %s  , %0.0f min'%(event_id, (toc-tic)/60))

            # Prepare submission for an event
            submission = pd.DataFrame(columns=['event_id', 'hit_id', 'track_id'],
                data=np.column_stack(([event_id,]*len(hits), hits.hit_id.values, track_id))
            ).astype(int)
            submissions.append(submission)
            
            for i in range(8):
                submission = extend(submission,hits)
            
            
            submission.to_csv('../cache/sub3/%s.csv.gz'%event_id,
                                index=False, compression='gzip')

            #------------------------------------------------------
    if 1:

        event_ids = [ '%09d'%i for i in range(0,125) ]  #(0,125)
        submissions = []
        for i,event_id in enumerate(event_ids):
            submission  = pd.read_csv('../cache/sub3/%s.csv.gz'%event_id, compression='gzip')
            submissions.append(submission)

        
        # Create submission file
        submission = pd.concat(submissions, axis=0)
        submission.to_csv('../submissions/sub3/submission-0021.csv.gz',
                            index=False, compression='gzip')
        print(len(submission))

In [19]:
run_make_submission()

event_id : 000000000  , 6 min
event_id : 000000001  , 15 min
event_id : 000000002  , 23 min
event_id : 000000003  , 32 min
event_id : 000000004  , 41 min
event_id : 000000005  , 50 min
event_id : 000000006  , 58 min
event_id : 000000007  , 67 min
event_id : 000000008  , 77 min
event_id : 000000009  , 86 min
event_id : 000000010  , 95 min
event_id : 000000011  , 103 min
event_id : 000000012  , 112 min
event_id : 000000013  , 121 min
event_id : 000000014  , 130 min
event_id : 000000015  , 137 min
event_id : 000000016  , 145 min
event_id : 000000017  , 155 min
event_id : 000000018  , 163 min
event_id : 000000019  , 171 min
event_id : 000000020  , 178 min
event_id : 000000021  , 186 min
event_id : 000000022  , 196 min
event_id : 000000023  , 205 min
event_id : 000000024  , 214 min
event_id : 000000025  , 223 min
event_id : 000000026  , 231 min
event_id : 000000027  , 239 min
event_id : 000000028  , 247 min
event_id : 000000029  , 254 min
event_id : 000000030  , 262 min
event_id : 000000031